In [1]:
from sympy.physics.mechanics import dynamicsymbols, inertia
import sympy as sp, numpy as np
import sympy.physics.vector as vector
from sympy.algebras.quaternion import Quaternion
from sympy.utilities.lambdify import lambdastr
from sympy import symbols
import sys, os

from simupy import codegen
from simupy.matrices import construct_explicit_matrix

INCLUDE_NUTATION_PRECESSION = False # below is a prototype derivation of the equations of motion including nutation and precession but it is not currently supported.

def display_dict(d):
    for k,v in d.items():
        display(sp.Eq(k,v))

# Kinematics Problem setup


## Symbol Table

Symbols are defined in this cell with their definitions neary by in in-line comments or through explicit naming.


In [2]:
t = dynamicsymbols._t # time symbol

omega_p = symbols('omega_p') # planet rotation angular velocity

if INCLUDE_NUTATION_PRECESSION:
    # pci2pcf_rotation_matrix = construct_explicit_matrix('pci2pcf', 3,3)
    pci2pcf_rotation_matrix = sp.MatrixSymbol('pci2pcf', 3,3)

lamda_D, phi_D, h_D = dynamicsymbols('lamda_D phi_D h_D') 
# planetodetic longitude, planetodetic latitude, planetodetic altitude
# order selected to math erfa's gc2gd signature
# symbol choice to match NESC's "Fundamentals of Geodetic Kinematics"
# swapping _E (earth) to _D for generic planetoDetic


a, f = symbols('a f', real=True, nonnegative=True) # planet equitorial radius and flattening coefficient

# placeholder derivatives for local acceleration trimming
lamda_D_dot, phi_D_dot, h_D_dot = dynamicsymbols('lamdadot_D phidot_D hdot_D')

planetodetic_pos_derivs = {}
for orig, dotted in zip((lamda_D, phi_D, h_D,),
                        (lamda_D_dot, phi_D_dot)):
    planetodetic_pos_derivs[orig.diff(t)] = dotted
    

psi, theta, phi = dynamicsymbols('psi theta phi') # euler angles local to body : yaw pitch roll


V_N, V_E, V_D = V_NED = sp.Array(
    # relative velocity in local frame (NED) -- add winds to get aero
    dynamicsymbols('V_N V_E V_D')
)

W_N, W_E, W_D = W_NED = sp.Array(
    dynamicsymbols('W_N W_E W_D')
)

# inertial position, velocity, and acceleration of vehicle center of mass
# expressed in PCI frame
px, py, pz = inertial_pos_I = sp.Array(dynamicsymbols('p_x p_y p_z'))
vx, vy, vz = inertial_vel_I = sp.Array(dynamicsymbols('v_x v_y v_z'))
ax, ay, az = inertial_acc_I = sp.Array(dynamicsymbols('a_x a_y a_z'))



# planet fixed rectangular position coordinates
planet_fixed_position_x, planet_fixed_position_y, planet_fixed_position_z = planet_fixed_position = sp.Array(
    dynamicsymbols('ap_x ap_y ap_z')
)

planet_fixed_func_args = (t, lamda_D, phi_D, h_D)

# atmospheric properties
density, speed_of_sound, viscosity = dynamicsymbols('rho c_s mu')

# translational acceleration due to gravity
gx, gy, gz = g = sp.Array(dynamicsymbols('g_x g_y g_z'))


# non-unit quaternion component symbols to represent I to body B attitude
q = sp.Array(
    dynamicsymbols('q_0:4') 
)
qmag = sp.sqrt(sp.tensorcontraction(sp.tensorproduct(q, q), (0,1))) # magnitude of non-unit quaternion
qq = Quaternion(*(q/qmag)) # q as unit-quaternion Quaternion object
cq = dynamicsymbols('c_q') # scalar control variable for derivative of q. See Rucker 2018

# angular velocity of vehicle body B w.r.t. inertial in body-fixed coordinates
omega_X, omega_Y, omega_Z = omega = I_ang_vel_B = sp.Array(dynamicsymbols('omega_X omega_Y omega_Z'))

# angular acceleration of vehicle body B w.r.t. inertial in body-fixed coordinates
alpha_X, alpha_Y, alpha_Z = I_ang_acc_B = sp.Array(dynamicsymbols('alpha_X alpha_Y alpha_Z'))

# inertial translational acceleration EXCLUDING GRAVITY of vehicle expressed in body-fixed coordinates
# intended to capture vehicle dynamics based on vehicle configuration (aero, prop, other control effectors, etc)
# input to kinematics block
AX, AY, AZ = inertial_acc_B = sp.Array(dynamicsymbols('A_X A_Y A_Z'))

airspeed_in_B = sp.Array(dynamicsymbols('u_B v_B w_B')) # true airspeed components in body-fixed coordinates
V_T, alpha, beta = sp.Array(dynamicsymbols('V_T alpha beta')) # true airspeed magnitude, angles of attack and sideslip
NED_ang_vel_B = sp.Array(dynamicsymbols('p_B q_B r_B')) # components of angular velocity of the body in air in body-fixed coordinate system

kinematics_state = sp.Array((*inertial_pos_I, *q, *inertial_vel_I, *I_ang_vel_B))
kinematics_input = sp.Array((*inertial_acc_B, *I_ang_acc_B, cq))

kinematics_output = sp.Array((
   *kinematics_state, 

   # these are guidance and control feedback outputs, essentially
   lamda_D, phi_D, h_D, # planet-fixed planetodetic spherical position coordinates -- follow logic of planet_fixed_position -- can be used for navigation/guidance problems
   psi, theta, phi, # NED-to-body yaw pitch roll, if possible

   # what is needed for aerodynamics 
   density, speed_of_sound, viscosity, # speed of sound, viscosity, and true airspeed give the needed atmospheric conditions
   V_T, alpha, beta, # true air speed, angles of attack and sideslip
   *NED_ang_vel_B, # components of angular velocity of the body in air in body-fixed coordinate system for computing aerodynamic damping

   *V_NED, # planet-fixed velocities in local frame
   *W_NED, # local wind
))


## Symbolic function placeholders

These symbols represent numerical functions to assign particular variables in the codegeneration. This approach is used to allow for parameterized models and to take advantage of numerically efficient algorithms when appropriate, such as for the planetodetic problem.

In [3]:
kinematics_function_modules = {} # defines the namespace of the functions used in the Planet

# symbolic placeholder for planet centered fixed to planetodetic coordinates and back
if INCLUDE_NUTATION_PRECESSION:
    pci2pcf_rotation_matrix_function = symbols('pci2pcf', cls=sp.Function)(t)
    pci2pcf_rotation_matrix_function.shape = (3,3)
    kinematics_function_modules[pci2pcf_rotation_matrix_function] = 'self'
    
    pcf2pd = symbols('planetodetics.pcf2pd', cls=sp.Function)(*planet_fixed_position)
else:
    pcf2pd = symbols('planetodetics.pcf2pd', cls=sp.Function)(*inertial_pos_I)

pcf2pd.shape = (3,)

pd2pcf = symbols('planetodetics.pd2pcf', cls=sp.Function)(lamda_D, phi_D, h_D)
pd2pcf.shape = (3,)

kinematics_function_modules[pcf2pd] = 'self'


# function for acceleration due to gravity 
if not INCLUDE_NUTATION_PRECESSION:
    gravity_func = sp.symbols('gravity', cls=sp.Function)(*inertial_pos_I)
else:
    gravity_func = sp.symbols('gravity', cls=sp.Function)(*planet_fixed_position)
gravity_func.shape = (3,)
kinematics_function_modules[gravity_func] = 'self'

# function for wind model
winds_func = sp.symbols('winds', cls=sp.Function)(*planet_fixed_func_args)
winds_func.shape = (3,)
kinematics_function_modules[winds_func] = 'self'

# function for atmospheric model
atmosphere_func = symbols('atmosphere', cls=sp.Function)(*planet_fixed_func_args)
atmosphere_func.shape = (3,)
kinematics_function_modules[atmosphere_func] = 'self'


## Kinematic Relationship

Construct frames of reference, points, and kinematic relationships similar to Kane's method.

In [4]:
# planet centered Inertial
I = vector.frame.ReferenceFrame('I') 

# Planet-fixed reference frame
if INCLUDE_NUTATION_PRECESSION:
    P = I.orientnew('P', 'DCM', sp.Matrix(pci2pcf_rotation_matrix), '321') # planet centered, planet fixed
    P.set_ang_vel(I, omega_p*P.z)
else: 
    P = I.orientnew('P', 'Body', [omega_p*t, 0, 0], '321') # planet centered, planet fixed

# Local NED frame
NED = P.orientnew('NED', 'Body', [lamda_D, -phi_D-sp.pi/2, 0], '321') # local/NED on planet surface

# Define planet center as stationary
I0 = vector.Point('I_0') # planet center


# reference frame for vehicle body B
B = I.orientnew('B', 'quaternion', qq.args)

B.set_ang_vel(I, omega_X*B.x + omega_Y*B.y + omega_Z*B.z)


B_cm = vector.Point('B_cm') # center of mass of vehicle body B

I0.set_vel(I, 0)
I0.set_acc(I, 0)
I0.set_vel(P, 0)
I0.set_acc(P, 0)

B_cm.set_pos(I0, px*I.x + py*I.y + pz*I.z)

B_cm.set_vel(I, vx*I.x + vy*I.y + vz*I.z)
B_cm.set_vel(B, 0)
B_cm.set_acc(I, ax*I.x + ay*I.y + az*I.z)
B_cm.set_acc(B, 0)




# components of acceleration
translation_acceleration_due_to_gravity = gx*I.x + gy*I.y + gz*I.z # from the gravitational model
translation_acceleration_due_to_body = AX*B.x + AY*B.y + AZ*B.z # from the dynamics model


# Derive Equations of Motion

## Inertial translational position derivative

Inertial position derivative is a simple derivative in the inertial frame.

In [5]:
inertial_pos_I_deriv_dict = sp.solve(inertial_pos_I.diff(t) - inertial_vel_I, inertial_pos_I.diff(t))
display_dict(inertial_pos_I_deriv_dict)

Eq(Derivative(p_x(t), t), v_x(t))

Eq(Derivative(p_y(t), t), v_y(t))

Eq(Derivative(p_z(t), t), v_z(t))

## Quaternion derivative

The derivative for non-unit quaternion comes from [TODO: ADD CITATION]. It is straightforward from unit quaternion derivatives after a normalization. This expression includes a numerical control term $c_q$.

In [6]:
# quaternion kinematics
def Omega(ang_vel):
    return sp.Matrix(
        [
            [0, -ang_vel[0], -ang_vel[1], -ang_vel[2]],
            [ang_vel[0], 0, ang_vel[2], -ang_vel[1]],
            [ang_vel[1], -ang_vel[2], 0, ang_vel[0]],
            [ang_vel[2], ang_vel[1], -ang_vel[0], 0]
        ]
    )

qvec = sp.Matrix(q)
qdot_vec_expr = Omega(I_ang_vel_B)@qvec/2 + cq*qvec
qdotvec = sp.Matrix(q.diff(t))
q_deriv_dict = sp.solve(qdot_vec_expr - qdotvec, qdotvec)


display_dict(q_deriv_dict)

Eq(Derivative(q_0(t), t), c_q(t)*q_0(t) - omega_X(t)*q_1(t)/2 - omega_Y(t)*q_2(t)/2 - omega_Z(t)*q_3(t)/2)

Eq(Derivative(q_1(t), t), c_q(t)*q_1(t) + omega_X(t)*q_0(t)/2 - omega_Y(t)*q_3(t)/2 + omega_Z(t)*q_2(t)/2)

Eq(Derivative(q_2(t), t), c_q(t)*q_2(t) + omega_X(t)*q_3(t)/2 + omega_Y(t)*q_0(t)/2 - omega_Z(t)*q_1(t)/2)

Eq(Derivative(q_3(t), t), c_q(t)*q_3(t) - omega_X(t)*q_2(t)/2 + omega_Y(t)*q_1(t)/2 + omega_Z(t)*q_0(t)/2)

## Inertial translational velocity derivative

We assume that the `Vehicle` outputs the translational acceleration due non-gravitational forces and the gravitaty model incorporates the translational acceleration due to gravity.

In [7]:
translational_acceleration_in_I_vec = translation_acceleration_due_to_gravity + translation_acceleration_due_to_body
translational_acceleration_in_I_expr = translational_acceleration_in_I_vec.to_matrix(I).simplify()
inertial_vel_I_deriv_dict = sp.solve(translational_acceleration_in_I_expr - sp.Matrix(inertial_vel_I.diff(t)), inertial_vel_I.diff(t))

display_dict(inertial_vel_I_deriv_dict)
# TODO: simplify the gravity components to reduce number of multiply and add

Eq(Derivative(v_x(t), t), (A_X(t)*q_0(t)**2 + A_X(t)*q_1(t)**2 - A_X(t)*q_2(t)**2 - A_X(t)*q_3(t)**2 - 2*A_Y(t)*q_0(t)*q_3(t) + 2*A_Y(t)*q_1(t)*q_2(t) + 2*A_Z(t)*q_0(t)*q_2(t) + 2*A_Z(t)*q_1(t)*q_3(t) + g_x(t)*q_0(t)**2 + g_x(t)*q_1(t)**2 + g_x(t)*q_2(t)**2 + g_x(t)*q_3(t)**2)/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

Eq(Derivative(v_y(t), t), (2*A_X(t)*q_0(t)*q_3(t) + 2*A_X(t)*q_1(t)*q_2(t) + A_Y(t)*q_0(t)**2 - A_Y(t)*q_1(t)**2 + A_Y(t)*q_2(t)**2 - A_Y(t)*q_3(t)**2 - 2*A_Z(t)*q_0(t)*q_1(t) + 2*A_Z(t)*q_2(t)*q_3(t) + g_y(t)*q_0(t)**2 + g_y(t)*q_1(t)**2 + g_y(t)*q_2(t)**2 + g_y(t)*q_3(t)**2)/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

Eq(Derivative(v_z(t), t), (-2*A_X(t)*q_0(t)*q_2(t) + 2*A_X(t)*q_1(t)*q_3(t) + 2*A_Y(t)*q_0(t)*q_1(t) + 2*A_Y(t)*q_2(t)*q_3(t) + A_Z(t)*q_0(t)**2 - A_Z(t)*q_1(t)**2 - A_Z(t)*q_2(t)**2 + A_Z(t)*q_3(t)**2 + g_z(t)*q_0(t)**2 + g_z(t)*q_1(t)**2 + g_z(t)*q_2(t)**2 + g_z(t)*q_3(t)**2)/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

## Inertial angular velocity derivative

The derivative of the angular velocity of the vehicle body relative to the inertial frame expressed in body-fixed coordinates is a simple time-derivative in the inertial frame.

In [8]:
I_ang_vel_B_deriv_dict = sp.solve(I_ang_vel_B.diff(t) - I_ang_acc_B, I_ang_vel_B.diff(t))
display_dict(I_ang_vel_B_deriv_dict)

Eq(Derivative(omega_X(t), t), alpha_X(t))

Eq(Derivative(omega_Y(t), t), alpha_Y(t))

Eq(Derivative(omega_Z(t), t), alpha_Z(t))

## Combine and prepare for codegen

We combine the derivative dictionaries into the `kinematics_state_equation_dict` to provide expressions for the state derivative function to return (the "expression dictionary" for the function to be implemented by codegen). We leverage the fact that Python 3.6+ has ordered dictionaries to the "extra-assignments" dictionary `kinematics_state_extra_dict` for assigning values prior to CSE.

In [9]:
kinematics_state_equation_dict = {}
for deriv_dict in inertial_pos_I_deriv_dict, inertial_vel_I_deriv_dict, q_deriv_dict, I_ang_vel_B_deriv_dict:
    kinematics_state_equation_dict.update(deriv_dict)

kinematics_state_extra_dict = {}
# If using nutation/precession, would need to compute planet fixed rectangular position before calling gravity
kinematics_state_extra_dict[g] = gravity_func
    

# Kinematic Outputs

Next we construct the additional outputs that are used in the `Vehicle` models for common flight vehicles. The `kinematics_output_dict` is expression dictionary and `kinematics_output_equation_extra_assinments_dict` is the extra-assignments dictionary.

## Initialize with planetodetics, gravity, and winds

In [10]:
kinematics_output_dict = {}
kinematics_output_equation_extra_assinments_dict = {}

# output all of the kinematic state veriables
for state_var in kinematics_state:
    kinematics_output_dict[state_var] = state_var

if INCLUDE_NUTATION_PRECESSION:
    # compute the rotation matrix and planet-fixed position in rectangular coordinates
    kinematics_output_equation_extra_assinments_dict[pci2pcf_rotation_matrix] = pci2pcf_rotation_matrix_function
    kinematics_output_equation_extra_assinments_dict[planet_fixed_position] = pci2pcf_rotation_matrix_function * inertial_pos_I
    
# use the planetodesy model to compute longitude, latitude, and altitude
kinematics_output_equation_extra_assinments_dict[sp.Array((lamda_D, phi_D, h_D))] = pcf2pd


if not INCLUDE_NUTATION_PRECESSION:
    # without nutation and precession, the side-reel time is accounted for by shifting the latitude
    kinematics_output_equation_extra_assinments_dict[lamda_D] = lamda_D - omega_p*t

kinematics_output_dict[lamda_D] = lamda_D
kinematics_output_dict[phi_D] = phi_D
kinematics_output_dict[h_D] = h_D

# wind model assumed to depend on the planetodetic position
kinematics_output_equation_extra_assinments_dict[W_NED] = winds_func



## Calculate local-to-body euler angles

Here we compute the euler angles roll ($\phi$), pitch ($\theta$), and yaw ($\phi$). We construct the local NED to body rotation matrix, then determine the euler angles following "Review of Attitude Representations Used for Aircraft Kinematics" by Phillips et. al., 2001.

In [11]:
NED_rotation_B = NED.dcm(I) @ I.dcm(B) # equivalent to NED.dcm(B), 
NED_rotation_B_simplified = NED_rotation_B.simplify()

# Phillips eq. 1 and 25 defines Rotatation matrix from inertia to body
# NED_rotation_B is body to NED
# this follows Eq. 53 from Phillips.

kinematics_output_dict[psi] = sp.atan2(NED_rotation_B_simplified[1,0], NED_rotation_B_simplified[0,0])
kinematics_output_dict[theta] = sp.asin(-NED_rotation_B_simplified[2,0]) 
kinematics_output_dict[phi] = sp.atan2(NED_rotation_B_simplified[2,1], NED_rotation_B_simplified[2,2])



## Atmosphere models

In [12]:
kinematics_output_equation_extra_assinments_dict[sp.Array([density, speed_of_sound, viscosity,])] = atmosphere_func

kinematics_output_dict[density] = density
kinematics_output_dict[speed_of_sound] = speed_of_sound
kinematics_output_dict[viscosity] = viscosity

## Computing elements for aerodynamics


Here, we calculate the wind experienced on the body by determining the velocity of the of the vehicle's center of mass in the planet-fixed frame $P$, incorporating the winds, and expressing the computed wind in body-fixed coordinates.

In [13]:
B_cm.v1pt_theory(I0, P, I) # calculate the velocity of the vehicle's center of mass in the planet-fixed frame
local_wind = W_N*NED.x + W_E*NED.y + W_D*NED.z
body_wind = B_cm.vel(P) - local_wind # incorporate the wind model
body_wind_I = body_wind.to_matrix(I).simplify() # express in inertial coordinates

body_wind_B = B.dcm(I).simplify()@body_wind_I # express the winds in body-fixed coordinates
body_wind_B_simplified = body_wind_B.simplify() # simplify

body_wind_B_simplified

Matrix([
[(2*(-q_0(t)*q_2(t) + q_1(t)*q_3(t))*(W_D(t)*sin(phi_D(t)) - W_N(t)*cos(phi_D(t)) + v_z(t)) + 2*(q_0(t)*q_3(t) + q_1(t)*q_2(t))*(-omega_p*p_x(t) + W_D(t)*sin(omega_p*t + lamda_D(t))*cos(phi_D(t)) - W_E(t)*cos(omega_p*t + lamda_D(t)) + W_N(t)*sin(omega_p*t + lamda_D(t))*sin(phi_D(t)) + v_y(t)) + (q_0(t)**2 + q_1(t)**2 - q_2(t)**2 - q_3(t)**2)*(omega_p*p_y(t) + W_D(t)*cos(omega_p*t + lamda_D(t))*cos(phi_D(t)) + W_E(t)*sin(omega_p*t + lamda_D(t)) + W_N(t)*sin(phi_D(t))*cos(omega_p*t + lamda_D(t)) + v_x(t)))/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2)],
[(2*(q_0(t)*q_1(t) + q_2(t)*q_3(t))*(W_D(t)*sin(phi_D(t)) - W_N(t)*cos(phi_D(t)) + v_z(t)) + 2*(-q_0(t)*q_3(t) + q_1(t)*q_2(t))*(omega_p*p_y(t) + W_D(t)*cos(omega_p*t + lamda_D(t))*cos(phi_D(t)) + W_E(t)*sin(omega_p*t + lamda_D(t)) + W_N(t)*sin(phi_D(t))*cos(omega_p*t + lamda_D(t)) + v_x(t)) + (q_0(t)**2 - q_1(t)**2 + q_2(t)**2 - q_3(t)**2)*(-omega_p*p_x(t) + W_D(t)*sin(omega_p*t + lamda_D(t))*cos(phi_D(t)) - W_E(t)*cos(omega_p

Next we compute the total true wind $V_T$ (the magnitude of the wind calculated above) and the angles of attack and sideslip.

In [14]:
V_T_simp = (body_wind.magnitude()**2).simplify()
kinematics_output_dict[V_T] = sp.sqrt(V_T_simp*sp.Heaviside(V_T_simp))
kinematics_output_dict[alpha] = sp.atan2(body_wind_B_simplified[2], body_wind_B_simplified[0])
kinematics_output_dict[beta] = sp.asin(
    sp.Piecewise( # This piecewise expression is used to provide the correct output at singularities
        (1., (body_wind_B_simplified[1]/kinematics_output_dict[V_T])>1.0), 
        (-1., (body_wind_B_simplified[1]/kinematics_output_dict[V_T])<-1.0),
        (body_wind_B_simplified[1]/kinematics_output_dict[V_T], kinematics_output_dict[V_T]>0.),
        (0., True)
    )
)

## Translational velocity in local NED frame

Here, we calculate the local velocity (in the planet centered planet fixed frame $P$)

In [15]:
B_cm.v1pt_theory(I0, P, I)
V_NED_expr = B_cm.vel(P).to_matrix(NED)
V_NED_expr_dict = sp.solve(V_NED_expr - sp.Matrix(V_NED), V_NED)

V_NED_cse = sp.cse(V_NED_expr, symbols=sp.numbered_symbols(prefix='V_NED_'), order='none')
V_NED_cse_assignments = {k: v for k,v in V_NED_cse[0]}
V_NED_cse_assignments.update({k: v for k, v in zip(V_NED, V_NED_cse[1][0])})
kinematics_output_equation_extra_assinments_dict.update(V_NED_cse_assignments)

display_dict(V_NED_expr_dict)

Eq(V_N(t), omega_p*p_x(t)*sin(omega_p*t + lamda_D(t))*sin(phi_D(t)) - omega_p*p_y(t)*sin(phi_D(t))*cos(omega_p*t + lamda_D(t)) - v_x(t)*sin(phi_D(t))*cos(omega_p*t + lamda_D(t)) - v_y(t)*sin(omega_p*t + lamda_D(t))*sin(phi_D(t)) + v_z(t)*cos(phi_D(t)))

Eq(V_E(t), -omega_p*p_x(t)*cos(omega_p*t + lamda_D(t)) - omega_p*p_y(t)*sin(omega_p*t + lamda_D(t)) - v_x(t)*sin(omega_p*t + lamda_D(t)) + v_y(t)*cos(omega_p*t + lamda_D(t)))

Eq(V_D(t), omega_p*p_x(t)*sin(omega_p*t + lamda_D(t))*cos(phi_D(t)) - omega_p*p_y(t)*cos(omega_p*t + lamda_D(t))*cos(phi_D(t)) - v_x(t)*cos(omega_p*t + lamda_D(t))*cos(phi_D(t)) - v_y(t)*sin(omega_p*t + lamda_D(t))*cos(phi_D(t)) - v_z(t)*sin(phi_D(t)))

## Angular velocity in local NED frame

Here we calculate the angular velocity components of the vehicle in air, used for the dynamic (or damping) aerodynamic moment derivatives.

In [16]:
e_squared = 1-(1-f)**2 # eccentricity squared
# radius in prime meridian
rho_M_expr = a*(1-e_squared)/sp.sqrt(1-e_squared*sp.sin(phi_D)**2)**(3)
# radius in prime vertical
rho_V_expr = a/sp.sqrt(1-e_squared*sp.sin(phi_D)**2)

navigation_kinematics = {
    phi_D_dot: sp.Piecewise((V_N /(rho_M_expr + h_D), a>0), (0., True)),
    lamda_D_dot:  sp.Piecewise((V_E /((rho_V_expr + h_D)*sp.cos(phi_D)), a>0), (0., True)),
    h_D_dot: -V_D
}

display_dict(navigation_kinematics)

navigation_cse = sp.cse(sp.Matrix([navigation_kinematics[key] for key in navigation_kinematics]), symbols=sp.numbered_symbols(prefix='navigation_'), order='none')

navigation_cse_assignments = {k: v for k,v in navigation_cse[0]}
navigation_cse_assignments.update({k: v for k, v in zip(navigation_kinematics.keys(), navigation_cse[1][0])})
kinematics_output_equation_extra_assinments_dict.update(navigation_cse_assignments)


Eq(phidot_D(t), Piecewise((V_N(t)/(a*(1 - f)**2/(-(1 - (1 - f)**2)*sin(phi_D(t))**2 + 1)**(3/2) + h_D(t)), a > 0), (0.0, True)))

Eq(lamdadot_D(t), Piecewise((V_E(t)/((a/sqrt(-(1 - (1 - f)**2)*sin(phi_D(t))**2 + 1) + h_D(t))*cos(phi_D(t))), a > 0), (0.0, True)))

Eq(hdot_D(t), -V_D(t))

In [17]:
NED_ang_vel_B_expr_dict = sp.solve(B.ang_vel_in(NED).to_matrix(B) - sp.Matrix(NED_ang_vel_B), NED_ang_vel_B)
NED_ang_vel_B_expr_simplified_dict = {}
for k,v in NED_ang_vel_B_expr_dict.items():
    NED_ang_vel_B_expr_simplified_dict[k] = v.subs(planetodetic_pos_derivs)
kinematics_output_dict.update(NED_ang_vel_B_expr_simplified_dict)
display_dict(NED_ang_vel_B_expr_simplified_dict)

Eq(p_B(t), (2*omega_p*q_0(t)*q_2(t) - 2*omega_p*q_1(t)*q_3(t) + 2*lamdadot_D(t)*q_0(t)*q_2(t) - 2*lamdadot_D(t)*q_1(t)*q_3(t) + omega_X(t)*q_0(t)**2 + omega_X(t)*q_1(t)**2 + omega_X(t)*q_2(t)**2 + omega_X(t)*q_3(t)**2 - phidot_D(t)*q_0(t)**2*sin(omega_p*t + lamda_D(t)) + 2*phidot_D(t)*q_0(t)*q_3(t)*cos(omega_p*t + lamda_D(t)) - phidot_D(t)*q_1(t)**2*sin(omega_p*t + lamda_D(t)) + 2*phidot_D(t)*q_1(t)*q_2(t)*cos(omega_p*t + lamda_D(t)) + phidot_D(t)*q_2(t)**2*sin(omega_p*t + lamda_D(t)) + phidot_D(t)*q_3(t)**2*sin(omega_p*t + lamda_D(t)))/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

Eq(q_B(t), (-2*omega_p*q_0(t)*q_1(t) - 2*omega_p*q_2(t)*q_3(t) - 2*lamdadot_D(t)*q_0(t)*q_1(t) - 2*lamdadot_D(t)*q_2(t)*q_3(t) + omega_Y(t)*q_0(t)**2 + omega_Y(t)*q_1(t)**2 + omega_Y(t)*q_2(t)**2 + omega_Y(t)*q_3(t)**2 + phidot_D(t)*q_0(t)**2*cos(omega_p*t + lamda_D(t)) + 2*phidot_D(t)*q_0(t)*q_3(t)*sin(omega_p*t + lamda_D(t)) - phidot_D(t)*q_1(t)**2*cos(omega_p*t + lamda_D(t)) - 2*phidot_D(t)*q_1(t)*q_2(t)*sin(omega_p*t + lamda_D(t)) + phidot_D(t)*q_2(t)**2*cos(omega_p*t + lamda_D(t)) - phidot_D(t)*q_3(t)**2*cos(omega_p*t + lamda_D(t)))/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

Eq(r_B(t), (-omega_p*q_0(t)**2 + omega_p*q_1(t)**2 + omega_p*q_2(t)**2 - omega_p*q_3(t)**2 - lamdadot_D(t)*q_0(t)**2 + lamdadot_D(t)*q_1(t)**2 + lamdadot_D(t)*q_2(t)**2 - lamdadot_D(t)*q_3(t)**2 + omega_Z(t)*q_0(t)**2 + omega_Z(t)*q_1(t)**2 + omega_Z(t)*q_2(t)**2 + omega_Z(t)*q_3(t)**2 - 2*phidot_D(t)*q_0(t)*q_1(t)*cos(omega_p*t + lamda_D(t)) - 2*phidot_D(t)*q_0(t)*q_2(t)*sin(omega_p*t + lamda_D(t)) - 2*phidot_D(t)*q_1(t)*q_3(t)*sin(omega_p*t + lamda_D(t)) + 2*phidot_D(t)*q_2(t)*q_3(t)*cos(omega_p*t + lamda_D(t)))/(q_0(t)**2 + q_1(t)**2 + q_2(t)**2 + q_3(t)**2))

Here we assign the local velocity and wind components in the local NED frame in the specified order.

In [18]:
for vel_var in V_NED:
    kinematics_output_dict[vel_var] = vel_var

# add the wind model outputs at the end to match the specified order
for wind_var in W_NED:
    kinematics_output_dict[wind_var] = wind_var

# Computing inertial initial conditions

Here we write a helper function that can compute the inertial initial conditions 

In [19]:
B2 = NED.orientnew('B2', 'Body', [psi, theta, phi], '321')
P_rotation_B = P.dcm(B2).simplify() # should be equivalent to LaRC slides [T^{BP}] at t=0, and taking into account PCI --> NED frames
# LaRC provides a formula that seems to be the special case of Phillips 2001 for q0**2  > q1**2, q2**2, q3**2, with some sign flips
# I'll stick with LaRC slides, but we should be aware of it based on



half_root_trace_plus_1 = sp.sqrt(P_rotation_B.trace()+1)/2
quaternion_from_spec = sp.Array([
    half_root_trace_plus_1,
    (P_rotation_B[2,1] - P_rotation_B[1,2])/(4*half_root_trace_plus_1),
    (P_rotation_B[0,2] - P_rotation_B[2,0])/(4*half_root_trace_plus_1),
    (P_rotation_B[1,0] - P_rotation_B[0,1])/(4*half_root_trace_plus_1),
    
])
quaternion_from_euler_dict = sp.solve(quaternion_from_spec- q, q)

inertial_trans_vel_from_relative_vel_NED_dict = sp.solve(B_cm.vel(P).to_matrix(NED) - sp.Matrix(V_NED), inertial_vel_I)

inertial_ang_vel_from_local = sp.solve(B.ang_vel_in(NED).to_matrix(B).subs(planetodetic_pos_derivs) - sp.Matrix(NED_ang_vel_B), I_ang_vel_B)

ic_from_pd_dict = {}
ic_from_pd_dict.update(inertial_trans_vel_from_relative_vel_NED_dict)
ic_from_pd_dict.update(quaternion_from_euler_dict)
ic_from_pd_dict.update(inertial_ang_vel_from_local)
ic_from_pd = kinematics_state.subs(ic_from_pd_dict).subs(omega_p*t, 0)

sp.physics.mechanics.functions.find_dynamicsymbols(quaternion_from_spec)
ic_extra = {}
ic_extra[inertial_pos_I] = pd2pcf
ic_extra.update(navigation_cse_assignments)
ic_from_pd_args = (lamda_D, phi_D, h_D, V_N, V_E, V_D, psi, theta, phi, *NED_ang_vel_B)


# Compute local trim residual

In order to trim the vehicle, we must calculate the acceleration of the vehicle's center of mass in terms of the local acceleration $\dot{V}_N$, $\dot{V}_E$, $\dot{V}_D$. The vehicle is trim whent he local acceleration is zero.

In [20]:
V_NED_vec = np.sum(np.array(V_NED)*np.array([NED.x, NED.y, NED.z]))
V_NED_dot_vec = np.sum(np.array(V_NED.diff(t))*np.array([NED.x, NED.y, NED.z]))

B_cm.set_vel(P, V_NED_vec)
V_NED_dot_residual = sp.Array(
    (translational_acceleration_in_I_vec - B_cm.a1pt_theory(I0, I, P)).simplify().to_matrix(NED).subs(planetodetic_pos_derivs) + V_NED_dot_vec.to_matrix(NED)
).reshape(3).subs(omega_p*t, 0)

In [21]:
trim_residual_extras = {}
trim_residual_extras[sp.Array((lamda_D, phi_D, h_D))] = pcf2pd
trim_residual_extras[g] = gravity_func

V_NED_cse_no_time = sp.cse(V_NED_expr.subs(omega_p*t, 0), symbols=sp.numbered_symbols(prefix='V_NED_'), order='none')
trim_residual_extras.update({k: v for k,v in V_NED_cse_no_time[0]})
trim_residual_extras.update({k: v for k, v in zip(V_NED, V_NED_cse_no_time[1][0])})
trim_residual_extras.update(navigation_cse_assignments)


# Kinematic codegen

In [22]:
transformation_func_to_print = [
    dict(num_name='inertial_to_body_dcm', input_args=q, 
        sym_expr=sp.Array(B.dcm(I).simplify()),
        extra_assignments={}, ),
    
    dict(num_name='body_to_NED_dcm', input_args=sp.Array([phi, theta, psi]), 
        sym_expr=sp.Array(B2.dcm(NED).simplify()),
        extra_assignments={}, ),

]

transformation_printer = codegen.ModuleNumPyPrinter(
    for_class=False, 
)

transformation_evaluator = codegen.ModulePrinter(transformation_printer)

transformation_print_params = codegen.process_funcs_to_print(transformation_func_to_print)

transformation_code = '\n'.join(transformation_evaluator.codeprint(transformation_print_params).split('\n')[2:])

In [23]:
kin_funcs_to_print = [    
    dict(num_name='kinematics_state_function', input_args=sp.Array([t, *kinematics_state, *kinematics_input]), 
        sym_expr=kinematics_state.diff(t).subs(kinematics_state_equation_dict),
        extra_assignments=kinematics_state_extra_dict ),
        
    dict(num_name='ic_from_planetodetic', input_args=sp.Array([*ic_from_pd_args]), 
        sym_expr=ic_from_pd,
        extra_assignments=ic_extra, ),
    
    dict(num_name='kinematics_output_function', input_args=sp.Array([t, *kinematics_state]), 
        sym_expr=kinematics_output.subs(kinematics_output_dict),
        extra_assignments=kinematics_output_equation_extra_assinments_dict, ),
    
    dict(num_name='inertial_to_NED_dcm', input_args=sp.Array([t, lamda_D, phi_D]), 
        sym_expr=sp.Array(NED.dcm(I).simplify()),
        extra_assignments={}, ),
    
    dict(num_name='local_translational_trim_residual', input_args=sp.Array([*kinematics_state[:-3], *kinematics_input[:3]]), 
        sym_expr=V_NED_dot_residual,
        extra_assignments=trim_residual_extras, ),
    
    
]

static_kinematics_function_modules = {}
for key in kinematics_function_modules:
    static_kinematics_function_modules[key.func] = kinematics_function_modules[key]

constant_modules = {omega_p: 'self', a: 'self', f: 'self'}
constant_names= {omega_p: 'planetodetics.'+str(omega_p), a: 'planetodetics.'+str(a), f: 'planetodetics.'+str(f)}

kin_code_print_params = codegen.process_funcs_to_print(kin_funcs_to_print)
kin_printer = codegen.ModuleNumPyPrinter(
    for_class=True,
    function_modules=static_kinematics_function_modules,# function_names=func_names,
    constant_modules=constant_modules, 
    constant_names = constant_names,
)
kin_evaluator = codegen.ModulePrinter(kin_printer)
kin_code = kin_evaluator.codeprint(
    (kin_code_print_params)
)
with open(os.path.join('..', 'simupy_flight', 'kinematics.py'), "w") as open_file:
    open_file.write(kin_code)
    open_file.write(transformation_code)

# Dynamics Modeling


Much of the effort around the `Vehicle` design is to provide sufficient API "hooks" to facilitate modeling of different vehicles without re-computing (or even re-`connect`ing, in the context of SimuPy) a cumbersome number of variables. To that end, we provide hooks for extra forces and moments directly as well as extra aerodynamic coefficients. Since the mathematical modeling is simpler but with several decompositions for the purposes of API, this section has fewer distinct sections and more and more explicitly named variables.

## Constants, Atmospheric flight condition, and other setup

Here we define the constants used in the inertia and aerodynamic models. We also label and define atmospheric quantities used for the aerodynamics modeling. The dynamic pressure $\bar{q}$ and Mach number $M$ could be calculated in the Kinematics model, however since Reynolds number $R_e$ also depends on the density and a vehicle-specific parameter, I kept these functions of the flight condition as part of the Dynamics model.

The dynamics model expression dictionary is `dynamics_output_dict` and the extra-assignments dictionary is `dynamics_output_extra_dict`.

In [24]:
dynamics_output_dict = {}
dynamics_output_extra_dict = {}

# Inertia symbols
m = symbols('m') 
Ixx, Iyy, Izz, Ixy, Iyz, Ixz = inertia_symbols = \
    symbols('I_xx, I_yy, I_zz, I_xy, I_yz, I_xz')
com_x, com_y, com_z = com_pos = sp.Array(sp.symbols('x_com y_com z_com'))

# Aerodynamic parameters
mrc_x, mrc_y, mrc_z = mrc_pos = sp.Array(sp.symbols('x_mrc y_mrc z_mrc'))
S_A, a_l, b_l, c_l, d_l = sp.symbols('S_A a_l b_l c_l d_l') # reference area; reference directional, lateral, longitudinal lengths; Reynolds number reference length
ref_lengths = np.array([a_l, c_l, b_l])

inertia_constants = (m, 
                   Ixx, Iyy, Izz, Ixy, Iyz, Ixz,
                   com_x, com_y, com_z)
aerodynamics_constants =  (mrc_x, mrc_y, mrc_z,
                   S_A, a_l, b_l, c_l, d_l)
dynamics_constants = (*inertia_constants, *aerodynamics_constants)

# atmospheric parameters
qbar, mach, reynolds = sp.symbols('qbar Ma Re')

qbar_expr = V_T**2*density/2
mach_expr = V_T/speed_of_sound
Re_expr = V_T*d_l*density/viscosity

dynamics_output_extra_dict[qbar] = qbar_expr
dynamics_output_extra_dict[mach] = mach_expr
dynamics_output_extra_dict[reynolds] = Re_expr

starargs = symbols('*args')



## Defining aerodynamic coefficients and their decompositions

In [25]:
# Wind frame, freestream wind in -W.x direction -- potentially 
W = vector.frame.ReferenceFrame('W')
# Stability frame, Sf.x and Sf.z in same plane as B.x, B.z
Sf = W.orientnew('Sf', 'Body', [-beta, 0, 0], '312')
Bw = Sf.orientnew('Bw', 'Body', [alpha, 0, 0], '231') # Body frame relative to wind

aero_forces = sp.Array(dynamicsymbols('F_ax F_ay F_az')) # aerodynamic forces in body-fixed coordinate system
Lcal, Mcal, Ncal = aero_moments = sp.Array(dynamicsymbols('Lcal Mcal Ncal')) # aerodynamic Roll, Pitch, Yaw moments (about body-fixed coordiante system)

# total aero coefficients
CD, CS, CL = total_aero_force_coeffs = sp.Array(dynamicsymbols('CD CS CL')) # static aero forces
CLcal, CMcal, CNcal = total_aero_moment_coeffs = sp.Array(dynamicsymbols('CLcal CMcal CNcal')) # static aero moments
# CLcal_s, CMcal_s, CNcal_s = transformed_static_aero_moment_coeffs = sp.Array(dynamicsymbols('CLcal_s CMcal_s CNcal_s')) # static aero moments
Cp, Cq, Cr = total_aero_damping_coeffs = sp.Array(dynamicsymbols('Cp Cq Cr')) # dynamic aero moments

# base aero coefficients
CDb, CSb, CLb = base_aero_force_coeffs =  np.array(dynamicsymbols('CD_b CS_b CL_b'))
CLcalb, CMcalb, CNcalb = base_aero_moment_coeffs = np.array(dynamicsymbols('CLcal_b CMcal_b CNcal_b'))
Cpb, Cqb, Crb = base_aero_damping_coeffs = np.array(dynamicsymbols('Cp_b Cq_b Cr_b'))

base_aero_coeffs = sp.Array([
    *base_aero_force_coeffs,
    *base_aero_moment_coeffs,
    *base_aero_damping_coeffs,
])


# extra aero coefficients -- inputs, can be used to model aerodynamic control surfaces or do configuration management on aero (i.e., add wings or tail to base vehicle)
# assumed to use same references areas/lengths as base, and moments are about same MRC
CDe, CSe, CLe = input_aero_force_coeffs = np.array(dynamicsymbols('CD_e CS_e CL_e'))
CLcale, CMcale, CNcale = input_aero_moment_coeffs = np.array(dynamicsymbols('CLcal_e CMcal_e CNcal_e'))
Cpe, Cqe, Cre = input_aero_damping_coeffs = np.array(dynamicsymbols('Cp_e Cq_e Cr_e'))


input_aero_coeffs = sp.Array([
    *input_aero_force_coeffs,
    *input_aero_moment_coeffs,
    *input_aero_damping_coeffs,
])




total_aero_force_coeffs = base_aero_force_coeffs + input_aero_force_coeffs
total_aero_damping_coeffs_mrc = base_aero_damping_coeffs + input_aero_damping_coeffs
total_static_aero_moment_coeffs_mrc = base_aero_moment_coeffs + input_aero_moment_coeffs 
total_aero_moment_coeffs_mrc = total_static_aero_moment_coeffs_mrc + total_aero_damping_coeffs_mrc*ref_lengths*NED_ang_vel_B*sp.Piecewise((1/(2*V_T), V_T> 0.), (0., True))

## Total Aerodynamic Forces and Moments

For the aerodynamic forces, we must transform the wind-frame Lift, Drag, and Side-forces to body-fixed Normal, Axial, and Side-forces. Note that these two side-forces do not point in the same direction for non-zero sideslip.

For the aerodynamic moments, we assume the moment coefficients are about a moment reference frame (MRC) so the moment coefficients must be transformed to the center of mass position. Note that we apply the same transformation to the damping coefficients.

In [26]:
body_unit_vecs = np.array([Bw.x, Bw.y, Bw.z])
wind_unit_vecs = np.array([W.x, W.y, W.z])


# force transformation
total_aero_force_coeffs_vec = np.sum(np.array([-1, 1, -1])*total_aero_force_coeffs*wind_unit_vecs)
total_aero_forces_expr = qbar*S_A*np.array(total_aero_force_coeffs_vec.to_matrix(Bw)).squeeze()

# moment transformation
total_aero_moment_coeffs_mrc_vec = np.sum(total_aero_moment_coeffs_mrc*body_unit_vecs)
mrc_to_com = np.array(com_pos - mrc_pos)
mrc_to_com_vec = np.sum(mrc_to_com*body_unit_vecs)
total_aero_moment_coeffs_com = np.array(total_aero_moment_coeffs_mrc - np.array((total_aero_force_coeffs_vec.cross(mrc_to_com_vec)).to_matrix(Bw)).squeeze()/ref_lengths)
total_aero_moments_com_expr = qbar*S_A*total_aero_moment_coeffs_com*ref_lengths


# in general, moment coefficient = static coefficient + ((dynamic coefficient * angular rate * ref length) / (2 * true airspeed))
# LaRC slides show an extra half factor on dynamic term, but NESC description doesn't have that... coefficients can be re-normalized
# so just need to clarify the definition


## Preparing for Codegen

In [27]:
# 
total_aero_forces_moments_syms = sp.Array([*aero_forces, *aero_moments])
total_aero_forces_moments_expr = sp.Array([*total_aero_forces_expr, *total_aero_moments_com_expr])



total_aero_forces_moments_func = sp.Function('_tot_aero_forces_moments')(qbar, mach, reynolds, V_T, alpha, beta, *NED_ang_vel_B, starargs)
total_aero_forces_moments_func.shape = (6,)
dynamics_output_extra_dict[total_aero_forces_moments_syms] = total_aero_forces_moments_func
tot_aero_extra_dict = {}

base_aero_coeffs_func = sp.Function('base_aero_coeffs')(alpha, beta, mach, reynolds)
base_aero_coeffs_func.shape = (9,)
tot_aero_extra_dict[base_aero_coeffs] = base_aero_coeffs_func

input_aero_coeffs_func = sp.Function('input_aero_coeffs')(alpha, beta, mach, reynolds, starargs)
input_aero_coeffs_func.shape = (9,)
tot_aero_extra_dict[input_aero_coeffs] = input_aero_coeffs_func

# input forces and torques/moments in body-fixed coordinates (moments about center of mass)
Phix, Phiy, Phiz = input_forces = sp.Array(dynamicsymbols('Phi_x Phi_y Phi_z'))
taux, tauy, tauz = input_moments = sp.Array(dynamicsymbols('tau_x tau_y tau_z'))
input_forces_moments = sp.Array([*input_forces, *input_moments])

input_forces_moments_func = sp.Function('_input_force_moment')(t, *kinematics_output, qbar, mach, reynolds, starargs)
input_forces_moments_func.shape = (6,)
dynamics_output_extra_dict[input_forces_moments] = input_forces_moments_func


# total forces and moments
Fx, Fy, Fz = tot_body_fixed_forces_var = sp.Array(dynamicsymbols('F_x F_y F_z')) # total body-fixed forces
Mx, My, Mz = tot_body_fixed_moments_var = sp.Array(dynamicsymbols('M_x M_y M_z')) # total body-fixed moments



tot_forces_moments_syms = sp.Array([*tot_body_fixed_forces_var, *tot_body_fixed_moments_var])
tot_forces_moments_expr = total_aero_forces_moments_syms + input_forces_moments
dynamics_output_extra_dict[tot_forces_moments_syms] = tot_forces_moments_expr



dynamics_input = sp.Array([
    t,
    *kinematics_output,
    starargs,
])

dynamics_output = sp.Array([*inertial_acc_B, *I_ang_acc_B])
# TODO: make the angular and translational accelerations independent hooked functions that take (Mx My Mz) and (Fx Fy Fz), respectively.

# Dynamics output

The translational acceleration of the vehicle center of mass and the angular acceleration of the vehicle body about the center of mass is computed using the Newton-Euler equation. For translational acceleration, we simply sum the forces projected in the body-fixed axes and divide by the (assumed constant) mass $m$ as shown below. The angular acceleration for a constant inertia $\mathbb{I}$ is given by 

$$\left[\begin{matrix}\dot{\omega}_{x}\\
\dot{\omega}_{y}\\
\dot{\omega}_{z}
\end{matrix}\right]=\mathbb{I}^{-1}\left(\left[\begin{matrix}M_{x}\\
M_{y}\\
M_{z}
\end{matrix}\right]-\left[\begin{matrix}\omega_{x}\\
\omega_{y}\\
\omega_{z}
\end{matrix}\right]\times\mathbb{I}\cdot\left[\begin{matrix}\omega_{x}\\
\omega_{y}\\
\omega_{z}
\end{matrix}\right]\right)$$

Below, we see the operations of this vector-matrix notation carried out.

In [28]:
inertial_acc_B_dict = sp.solve(m*inertial_acc_B - tot_body_fixed_forces_var, inertial_acc_B) # this assumes constant mass
dynamics_output_dict.update(inertial_acc_B_dict)
display_dict(inertial_acc_B_dict)

Eq(A_X(t), F_x(t)/m)

Eq(A_Y(t), F_y(t)/m)

Eq(A_Z(t), F_z(t)/m)

In [29]:
II = inertia(Bw, Ixx, Iyy, Izz, -Ixy, -Iyz, -Ixz)
Bw_ang_vel_I = omega_X*Bw.x + omega_Y*Bw.y + omega_Z*Bw.z
Bw_ang_acc_I = alpha_X*Bw.x + alpha_Y*Bw.y + alpha_Z*Bw.z
inertial_ang_acc_B_dict = sp.solve(sp.Array((II.dot(Bw_ang_acc_I) + Bw_ang_vel_I.cross(II.dot(Bw_ang_vel_I))).to_matrix(Bw))[:, 0] - tot_body_fixed_moments_var, I_ang_acc_B)
dynamics_output_dict.update(inertial_ang_acc_B_dict)

display_dict(inertial_ang_acc_B_dict)

Eq(alpha_X(t), -((I_xy*I_yz + I_xz*I_yy)*(I_xx*omega_X(t)*omega_Y(t) + I_xy*omega_X(t)**2 - I_xy*omega_Y(t)**2 - I_xz*omega_Y(t)*omega_Z(t) - I_yy*omega_X(t)*omega_Y(t) + I_yz*omega_X(t)*omega_Z(t) + M_z(t)) + (I_xy*I_zz + I_xz*I_yz)*(-I_xx*omega_X(t)*omega_Z(t) + I_xy*omega_Y(t)*omega_Z(t) - I_xz*omega_X(t)**2 + I_xz*omega_Z(t)**2 - I_yz*omega_X(t)*omega_Y(t) + I_zz*omega_X(t)*omega_Z(t) + M_y(t)) + (I_yy*I_zz - I_yz**2)*(-I_xy*omega_X(t)*omega_Z(t) + I_xz*omega_X(t)*omega_Y(t) + I_yy*omega_Y(t)*omega_Z(t) + I_yz*omega_Y(t)**2 - I_yz*omega_Z(t)**2 - I_zz*omega_Y(t)*omega_Z(t) + M_x(t)))/(-I_xx*I_yy*I_zz + I_xx*I_yz**2 + I_xy**2*I_zz + 2*I_xy*I_xz*I_yz + I_xz**2*I_yy))

Eq(alpha_Y(t), -((I_xx*I_yz + I_xy*I_xz)*(I_xx*omega_X(t)*omega_Y(t) + I_xy*omega_X(t)**2 - I_xy*omega_Y(t)**2 - I_xz*omega_Y(t)*omega_Z(t) - I_yy*omega_X(t)*omega_Y(t) + I_yz*omega_X(t)*omega_Z(t) + M_z(t)) + (I_xx*I_zz - I_xz**2)*(-I_xx*omega_X(t)*omega_Z(t) + I_xy*omega_Y(t)*omega_Z(t) - I_xz*omega_X(t)**2 + I_xz*omega_Z(t)**2 - I_yz*omega_X(t)*omega_Y(t) + I_zz*omega_X(t)*omega_Z(t) + M_y(t)) + (I_xy*I_zz + I_xz*I_yz)*(-I_xy*omega_X(t)*omega_Z(t) + I_xz*omega_X(t)*omega_Y(t) + I_yy*omega_Y(t)*omega_Z(t) + I_yz*omega_Y(t)**2 - I_yz*omega_Z(t)**2 - I_zz*omega_Y(t)*omega_Z(t) + M_x(t)))/(-I_xx*I_yy*I_zz + I_xx*I_yz**2 + I_xy**2*I_zz + 2*I_xy*I_xz*I_yz + I_xz**2*I_yy))

Eq(alpha_Z(t), -((I_xx*I_yy - I_xy**2)*(I_xx*omega_X(t)*omega_Y(t) + I_xy*omega_X(t)**2 - I_xy*omega_Y(t)**2 - I_xz*omega_Y(t)*omega_Z(t) - I_yy*omega_X(t)*omega_Y(t) + I_yz*omega_X(t)*omega_Z(t) + M_z(t)) + (I_xx*I_yz + I_xy*I_xz)*(-I_xx*omega_X(t)*omega_Z(t) + I_xy*omega_Y(t)*omega_Z(t) - I_xz*omega_X(t)**2 + I_xz*omega_Z(t)**2 - I_yz*omega_X(t)*omega_Y(t) + I_zz*omega_X(t)*omega_Z(t) + M_y(t)) + (I_xy*I_yz + I_xz*I_yy)*(-I_xy*omega_X(t)*omega_Z(t) + I_xz*omega_X(t)*omega_Y(t) + I_yy*omega_Y(t)*omega_Z(t) + I_yz*omega_Y(t)**2 - I_yz*omega_Z(t)**2 - I_zz*omega_Y(t)*omega_Z(t) + M_x(t)))/(-I_xx*I_yy*I_zz + I_xx*I_yz**2 + I_xy**2*I_zz + 2*I_xy*I_xz*I_yz + I_xz**2*I_yy))

In [30]:


transformation_func_to_print = [
    dict(num_name='body_to_wind_dcm', input_args=sp.Array([alpha, beta]), 
        sym_expr=sp.Array(W.dcm(Bw)),
        extra_assignments={}, ),
    
]

transformation_printer = codegen.ModuleNumPyPrinter(
    for_class=False, 
)

transformation_evaluator = codegen.ModulePrinter(transformation_printer)

transformation_print_params = codegen.process_funcs_to_print(transformation_func_to_print)

transformation_code = '\n'.join(transformation_evaluator.codeprint(transformation_print_params).split('\n')[2:])


In [31]:
def CPM(vec):
    return sp.Array(
        [
            [0, -vec[2], vec[1],],
            [vec[2], 0, -vec[0],],
            [-vec[1], vec[0], 0,],
        ]
    )

In [32]:
dyn_funcs_to_print = [
    dict(num_name=total_aero_forces_moments_func.func.__name__[1:], input_args=sp.Array(total_aero_forces_moments_func.args), 
        sym_expr=total_aero_forces_moments_expr,
        extra_assignments=tot_aero_extra_dict, ),
    
    dict(num_name='dynamics_output_function', input_args=dynamics_input, 
        sym_expr=dynamics_output.subs(dynamics_output_dict),
        extra_assignments=dynamics_output_extra_dict, ),
    
    
    dict(num_name='mrc_to_com_cpm', input_args=[], 
        sym_expr=CPM(mrc_to_com_vec.to_matrix(Bw)),
        extra_assignments={}, ),

]

dyn_printer = codegen.ModuleNumPyPrinter(
    for_class=True, function_modules={total_aero_forces_moments_func.func: 'self', base_aero_coeffs_func.func: 'self',},
#     function_modules=func_modules, function_names=func_names,
    constant_modules={var: 'self' for var in dynamics_constants}
)
dyn_evaluator = codegen.ModulePrinter(dyn_printer)

dyn_code_print_params = codegen.process_funcs_to_print(dyn_funcs_to_print)

dyn_code = dyn_evaluator.codeprint(dyn_code_print_params) 

with open(os.path.join('..', 'simupy_flight', 'dynamics.py'), "w") as open_file:
    open_file.write(dyn_code)
    open_file.write(transformation_code)

Here we print plain-text column names, LaTeX column names, and a set of index constants to facilitate user development with the SimuPy Flight Vehicle Toolkit blocks.

In [33]:
print([str(arg) for arg in codegen.staticfy_expressions(kinematics_output)] + [str(arg) for arg in codegen.staticfy_expressions(dynamics_output)], "\n")
print(['$'+sp.latex(arg)+'$' for arg in codegen.staticfy_expressions(kinematics_output)] + ['$'+sp.latex(arg)+'$' for arg in codegen.staticfy_expressions(dynamics_output)],"\n")
for idx, arg in enumerate(codegen.staticfy_expressions(kinematics_output)):
    print("%s_idx = %d" % (str(arg), idx))
print("\n")
for idx, arg in enumerate(codegen.staticfy_expressions(sp.Array(input_forces_moments_func.args))):
    print("%s_arg_idx = %d" % (str(arg), idx))

['p_x', 'p_y', 'p_z', 'q_0', 'q_1', 'q_2', 'q_3', 'v_x', 'v_y', 'v_z', 'omega_X', 'omega_Y', 'omega_Z', 'lamda_D', 'phi_D', 'h_D', 'psi', 'theta', 'phi', 'rho', 'c_s', 'mu', 'V_T', 'alpha', 'beta', 'p_B', 'q_B', 'r_B', 'V_N', 'V_E', 'V_D', 'W_N', 'W_E', 'W_D', 'A_X', 'A_Y', 'A_Z', 'alpha_X', 'alpha_Y', 'alpha_Z'] 

['$p_{x}$', '$p_{y}$', '$p_{z}$', '$q_{0}$', '$q_{1}$', '$q_{2}$', '$q_{3}$', '$v_{x}$', '$v_{y}$', '$v_{z}$', '$\\omega_{X}$', '$\\omega_{Y}$', '$\\omega_{Z}$', '$\\lambda_{D}$', '$\\phi_{D}$', '$h_{D}$', '$\\psi$', '$\\theta$', '$\\phi$', '$\\rho$', '$c_{s}$', '$\\mu$', '$V_{T}$', '$\\alpha$', '$\\beta$', '$p_{B}$', '$q_{B}$', '$r_{B}$', '$V_{N}$', '$V_{E}$', '$V_{D}$', '$W_{N}$', '$W_{E}$', '$W_{D}$', '$A_{X}$', '$A_{Y}$', '$A_{Z}$', '$\\alpha_{X}$', '$\\alpha_{Y}$', '$\\alpha_{Z}$'] 

p_x_idx = 0
p_y_idx = 1
p_z_idx = 2
q_0_idx = 3
q_1_idx = 4
q_2_idx = 5
q_3_idx = 6
v_x_idx = 7
v_y_idx = 8
v_z_idx = 9
omega_X_idx = 10
omega_Y_idx = 11
omega_Z_idx = 12
lamda_D_idx = 13
p